## 44 - Modeling: Decision Tree

> One of the modeling candidates trials for actual modeling procedure.

**Decision Trees** (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation. In sklearn packages, the decision tree uses CART algorithm.

Some advantages of decision trees are:

- Simple to understand and to interpret. Trees can be visualised.

- Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.

- The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.

- Able to handle both numerical and categorical data. However scikit-learn implementation does not support categorical variables for now. Other techniques are usually specialised in analysing datasets that have only one type of variable. See algorithms for more information.

- Able to handle multi-output problems.

- Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.

- Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.

- Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.

The disadvantages of decision trees include:

- Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.

- Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.

- Predictions of decision trees are neither smooth nor continuous, but piecewise constant approximations as seen in the above figure. Therefore, they are not good at extrapolation.

- The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.

- There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.

- Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.

## Install needed files
**Graphviz - Step 1** Download at http://www.graphviz.org/download/

**Graphviz - Step 2** pip install graphviz

**Pydotplus** pip install pydotplus

In [ ]:
# import needed packages
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn import metrics
import graphviz
import pydotplus

In [ ]:
# read the data
full_data = pd.read_csv('data/cumulative_data_fe.csv')
train_data = pd.read_csv('data/train_data_fe.csv')
test_data = pd.read_csv('data/test_data_fe.csv')

# drop columns
y_train_data = pd.DataFrame(train_data['stone_soil_enc'])
x_train_data = train_data.drop(['id', 'img_id', 'stone_soil', 'stone_soil_enc'], axis = 1)

# drop columns
y_test_data = pd.DataFrame(test_data['stone_soil_enc'])
x_test_data = test_data.drop(['id', 'img_id', 'stone_soil', 'stone_soil_enc'], axis = 1)

In [ ]:
# define a function for scoring
def pred_metrics(real, pred) -> None:
    print("Accuracy:\t{}".format(metrics.accuracy_score(real, pred)))
    print("Precision:\t{}".format(metrics.precision_score(real, pred)))
    print("Recall:\t\t{}".format(metrics.recall_score(real, pred)))
    print("F1:\t\t{}".format(metrics.f1_score(real, pred)))

In [ ]:
# initialize DT classifier instance and train
clf = tree.DecisionTreeClassifier(max_depth=7, min_samples_leaf=7)
clf.fit(x_train_data, y_train_data)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=7)

**PS:** The max depth and min samples leaf here is default to avoid overfitting and wait for change.

In [ ]:
# calculate metrics of training
y_train_pred = clf.predict(x_train_data)
pred_metrics(y_train_data, y_train_pred)

Accuracy:	0.9655271108284306
Precision:	0.8319273199221285
Recall:		0.6084480303749407
F1:		0.7028508771929826


In [ ]:
# calculate metrics of testing
y_test_pred = clf.predict(x_test_data)
pred_metrics(y_test_data, y_test_pred)

Accuracy:	0.9528049866429208
Precision:	0.7314439946018894
Recall:		0.49953917050691243
F1:		0.5936473165388828


In [ ]:
# cross validation
cross_val_score(clf, x_train_data, y_train_data, cv= 10, scoring = 'f1')

array([0.6739427 , 0.7100271 , 0.7       , 0.65671642, 0.68344371,
       0.69986894, 0.67801047, 0.65578231, 0.65327979, 0.69095816])

In [ ]:
# find best parameters by grid search
paras = {"max_depth":[*range(3,16,3)], "min_samples_leaf":[*range(5,16,5)]}
gs = GridSearchCV(tree.DecisionTreeClassifier(), paras, cv=10, scoring='f1')
gs.fit(x_train_data, y_train_data)
print(gs.best_params_)
print(gs.best_score_)

{'max_depth': 9, 'min_samples_leaf': 5}
0.6950221026157586


In [ ]:
# train new model and cross validate
clf = tree.DecisionTreeClassifier(max_depth=9, min_samples_leaf=5)
clf.fit(x_train_data, y_train_data)
y_train_pred = clf.predict(x_train_data)
pred_metrics(y_train_data, y_train_pred)
cross_val_score(clf, x_train_data, y_train_data, cv= 10, scoring = 'f1')

Accuracy:	0.9704722531404039
Precision:	0.8622809714110052
Recall:		0.6656383483626008
F1:		0.7513057452792287


array([0.69448183, 0.72628726, 0.7114094 , 0.69425901, 0.71018277,
       0.7010582 , 0.67292225, 0.67567568, 0.64673913, 0.71821756])

In [ ]:
# test
y_test_pred = clf.predict(x_test_data)
pred_metrics(y_test_data, y_test_pred)

Accuracy:	0.9525505660857397
Precision:	0.7029940119760479
Recall:		0.5410138248847927
F1:		0.6114583333333334


In [ ]:
# visualize the model
dot = tree.export_graphviz(clf)
graph = pydotplus.graph_from_dot_data(dot)
graph.write_png("tree.png")

True